<a href="https://colab.research.google.com/github/CD-AC/Master_AI/blob/main/mia07_lab_red_clasica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ACTIVIDAD DE CLASIFICACIÓN DE TEXTO

En esta actividad vamos a trabajar en clasificar textos. Se recorrerá todo el proceso desde traer el dataset hasta proceder a dicha clasificación. Durante la actividad se llevarán a cabo muchos procesos como la creación de un vocabulario, el uso de embeddings y la creación de modelos.

Las cuestiones presentes en esta actividad están basadas en un Notebook creado por François Chollet, uno de los creadores de Keras y autor del libro "Deep Learning with Python".

En este Notebook se trabaja con el dataset "Newsgroup20" que contiene aproximadamente 20000 mensajes que pertenecen a 20 categorías diferentes.

El objetivo es entender los conceptos que se trabajan y ser capaz de hacer pequeñas experimentaciones para mejorar el Notebook creado.

# Librerías

In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Descarga de Datos

In [ ]:
data_path = keras.utils.get_file(
    "news20.tar.gz",
    "http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz",
    untar=True,
)

In [16]:
import os
import pathlib

!tar -xzf /content/news20.tar.gz -C /content

data_path = "/content"

print("Ruta definida:", data_path)
print("Contenido en data_path:\n", os.listdir(data_path))

Ruta definida: /content
Contenido en data_path:
 ['.config', 'news20.tar.gz', '20_newsgroup', 'sample_data']


In [17]:
import os
import pathlib

#Estructura de directorios del dataset
data_dir = pathlib.Path(data_path).parent / "/content/20_newsgroup"
dirnames = os.listdir(data_dir)
print("Number of directories:", len(dirnames))
print("Directory names:", dirnames)

Number of directories: 20
Directory names: ['rec.sport.hockey', 'sci.med', 'sci.crypt', 'talk.politics.misc', 'talk.religion.misc', 'comp.sys.mac.hardware', 'rec.motorcycles', 'comp.graphics', 'misc.forsale', 'sci.space', 'rec.autos', 'talk.politics.guns', 'talk.politics.mideast', 'comp.windows.x', 'comp.os.ms-windows.misc', 'rec.sport.baseball', 'alt.atheism', 'sci.electronics', 'soc.religion.christian', 'comp.sys.ibm.pc.hardware']


In [18]:
print(data_dir)

/content/20_newsgroup


In [19]:
#Algunos archivos de la categoria "com.graphics"
fnames = os.listdir(data_dir / "comp.graphics")
print("Number of files in comp.graphics:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in comp.graphics: 1000
Some example filenames: ['39669', '37922', '38379', '38225', '38544']


In [20]:
#Ejemplo de un texto de la categoría "com.graphics"
print(open(data_dir / "comp.graphics" / "37261").read())

Xref: cantaloupe.srv.cs.cmu.edu comp.graphics:37261 alt.graphics:519 comp.graphics.animation:2614
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!uwm.edu!zaphod.mps.ohio-state.edu!darwin.sura.net!dtix.dt.navy.mil!oasys!lipman
From: lipman@oasys.dt.navy.mil (Robert Lipman)
Newsgroups: comp.graphics,alt.graphics,comp.graphics.animation
Subject: CALL FOR PRESENTATIONS: Navy SciViz/VR Seminar
Message-ID: <32850@oasys.dt.navy.mil>
Date: 19 Mar 93 20:10:23 GMT
Article-I.D.: oasys.32850
Expires: 30 Apr 93 04:00:00 GMT
Reply-To: lipman@oasys.dt.navy.mil (Robert Lipman)
Followup-To: comp.graphics
Distribution: usa
Organization: Carderock Division, NSWC, Bethesda, MD
Lines: 65


			CALL FOR PRESENTATIONS
	
      NAVY SCIENTIFIC VISUALIZATION AND VIRTUAL REALITY SEMINAR

			Tuesday, June 22, 1993

	    Carderock Division, Naval Surface Warfare Center
	      (formerly the David Taylor Research Center)
			  Bethesda, Maryland

SPONSOR: NESS (Navy Engineering Software System) is sponsori

In [21]:
#Algunos archivos de la categoria "talk.politics.misc"
fnames = os.listdir(data_dir / "talk.politics.misc")
print("Number of files in talk.politics.misc:", len(fnames))
print("Some example filenames:", fnames[:5])

Number of files in talk.politics.misc: 1000
Some example filenames: ['178692', '178799', '179012', '177023', '178932']


In [23]:
#Ejemplo de un texto de la categoría "talk.politics.misc"
print(open(data_dir / "talk.politics.misc" / "178463").read())

Xref: cantaloupe.srv.cs.cmu.edu talk.politics.guns:54219 talk.politics.misc:178463
Newsgroups: talk.politics.guns,talk.politics.misc
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!magnus.acs.ohio-state.edu!usenet.ins.cwru.edu!agate!spool.mu.edu!darwin.sura.net!martha.utcc.utk.edu!FRANKENSTEIN.CE.UTK.EDU!VEAL
From: VEAL@utkvm1.utk.edu (David Veal)
Subject: Re: Proof of the Viability of Gun Control
Message-ID: <VEAL.749.735192116@utkvm1.utk.edu>
Lines: 21
Sender: usenet@martha.utcc.utk.edu (USENET News System)
Organization: University of Tennessee Division of Continuing Education
References: <1qpbqd$ntl@access.digex.net> <C5otvp.ItL@magpie.linknet.com>
Date: Mon, 19 Apr 1993 04:01:56 GMT

[alt.drugs and alt.conspiracy removed from newsgroups line.]

In article <C5otvp.ItL@magpie.linknet.com> neal@magpie.linknet.com (Neal) writes:

>   Once the National Guard has been called into federal service,
>it is under the command of the present. Tha N

In [24]:
list_all_dir = [
    'alt.atheism',
    'comp.graphics',
    'comp.sys.mac.hardware',
    'comp.windows.x',
    'misc.forsale',
    'rec.autos',
    'rec.sport.baseball',
    'rec.sport.hockey',
    'sci.crypt',
    'sci.med',
    'sci.space',
    'soc.religion.christian',
    'talk.politics.guns',
    'talk.politics.misc',
    'talk.religion.misc'
]

In [25]:
samples = []
labels = []
class_names = []
class_index = 0
for dirname in list_all_dir:
    class_names.append(dirname)
    dirpath = data_dir / dirname
    fnames = os.listdir(dirpath)
    print("Processing %s, %d files found" % (dirname, len(fnames)))
    for fname in fnames:
        fpath = dirpath / fname
        f = open(fpath, encoding="latin-1")
        content = f.read()
        lines = content.split("\n")
        lines = lines[10:]
        content = "\n".join(lines)
        samples.append(content)
        labels.append(class_index)
    class_index += 1

print("Classes:", class_names)
print("Number of samples:", len(samples))

Processing alt.atheism, 1000 files found
Processing comp.graphics, 1000 files found
Processing comp.sys.mac.hardware, 1000 files found
Processing comp.windows.x, 1000 files found
Processing misc.forsale, 1000 files found
Processing rec.autos, 1000 files found
Processing rec.sport.baseball, 1000 files found
Processing rec.sport.hockey, 1000 files found
Processing sci.crypt, 1000 files found
Processing sci.med, 1000 files found
Processing sci.space, 1000 files found
Processing soc.religion.christian, 997 files found
Processing talk.politics.guns, 1000 files found
Processing talk.politics.misc, 1000 files found
Processing talk.religion.misc, 1000 files found
Classes: ['alt.atheism', 'comp.graphics', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.misc', 'talk.religion.misc']
Number of samples: 14997


# Mezclando los datos para separarlos en Traning y Test

In [26]:
# Shuffle the data
seed = 1337
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)
keras.utils.set_random_seed(seed)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

# Tokenización de las palabras con TextVectorization

In [27]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples).batch(128)
vectorizer.adapt(text_ds)

In [29]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'the', 'to', 'of']

In [30]:
len(vectorizer.get_vocabulary())

20000

# Viendo la salida de Vectorizer

In [31]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 3762, 1955,   18,    2, 5188])

In [32]:
output

<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[   2, 3762, 1955,   18,    2, 5188,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   

In [33]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [34]:
test = ["the", "cat", "sat", "on", "the", "mat"]
[word_index[w] for w in test]

[2, 3762, 1955, 18, 2, 5188]

# Tokenización de los datos de entrenamiento y validación

In [35]:
x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

# Creación y entrenamiento del modelo. Red Neuronal Clásica

In [36]:
modeloClasico = keras.models.Sequential()
modeloClasico.add(keras.layers.Embedding(20000, 10, input_length=200))
modeloClasico.add(keras.layers.Flatten())
modeloClasico.add(keras.layers.Dense(512, activation='relu'))
modeloClasico.add(keras.layers.Dropout(0.3))
modeloClasico.add(keras.layers.Dense(20, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [37]:
modeloClasico.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modeloClasico.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"])
modeloClasico.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))
print(modeloClasico.summary())

Epoch 1/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 15s 99ms/step - acc: 0.0977 - loss: 2.7876 - val_acc: 0.1927 - val_loss: 2.4346
Epoch 2/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - acc: 0.2523 - loss: 2.2722 - val_acc: 0.3371 - val_loss: 1.9275
Epoch 3/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - acc: 0.4754 - loss: 1.6513 - val_acc: 0.4705 - val_loss: 1.5050
Epoch 4/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - acc: 0.6595 - loss: 1.1173 - val_acc: 0.5615 - val_loss: 1.2547
Epoch 5/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - acc: 0.7804 - loss: 0.7441 - val_acc: 0.6122 - val_loss: 1.1023
Epoch 6/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - acc: 0.8585 - loss: 0.5023 - val_acc: 0.6399 - val_loss: 1.0426
Epoch 7/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - acc: 0.8983 - loss: 0.3593 - val_acc: 0.6642 - val_loss: 1.0164
Epoch 8/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - acc: 0.9211 - loss: 0.2696 - val_acc: 0.6712 - val_loss: 1.0251
Epoch 9/20
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - acc: 0.930

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 200, 10)             │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 2000)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │       1,024,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 20)                  │          10,260 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,469,546 (9.42 MB)

 Trainable params: 1,234,772 (4.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,234,774 (4.71 MB)

None


# Evaluación

In [40]:
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
preds = modeloClasico(x)
end_to_end_model = keras.Model(string_input, preds)

processed_input = vectorizer(["this message is about computer graphics and 3D modeling"])
probabilities = modeloClasico.predict(processed_input)

class_names[np.argmax(probabilities[0])]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step


'comp.graphics'

In [41]:
processed_input = vectorizer(["politics and federal courts law that people understand with politician and elects congressman"])
probabilities = modeloClasico.predict(processed_input)

class_names[np.argmax(probabilities[0])]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


'talk.religion.misc'

In [42]:
processed_input = vectorizer(["we are talking about religion"])
probabilities = modeloClasico.predict(processed_input)

class_names[np.argmax(probabilities[0])]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


'alt.atheism'